_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## INTRODUCTION

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

#### IMPORT LIBRARIES

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import plotly.express as px
import numpy as np
import pandas as pd
from scipy.io import arff
import shap
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datapane as dp
from sklearn.tree import DecisionTreeClassifier

#### IMPORT FUNCTIONS

In [2]:
#IMPORT FUNCTIONS

from functions.data_clean import transform_data_ACS, transform_data_adult, transform_data_bank,transform_databank_churns, scale_df
from functions.validation_tuner import tuner_df
from functions.csv_exporter import intermodel_to_csv
from functions.csv_exporter import intermodel_to_csv_two
from functions.graph_generator import box_graph_inter,  selection_comparison, data_sample_chart, number_of_individuals, disparity_sample_percetange, disparity_selection_percentage, data_selectionrate_chart
from functions.validation_tuner import classifiers_df
from functions.metrics_model import percentage_calc

#### IMPORT DATASETS

- The following functions transform and convert the selected datasets

In [3]:
#import datasets 

''' Adult dataset  ''' 

df_one =  X_raw, Y = shap.datasets.adult()
df_one = transform_data_adult(X_raw, Y)

#target = income abobe $50,000

''' ACS income ''' 

#df_two = transform_data_ACS((pd.DataFrame(arff.loadarff(r"Data\ACSIncome_state_number.arff")[0])))

#target = income abobe $50,000

''' Credit card churn  '''

#target = aprroved yes-no

df_three = transform_databank_churns(pd.read_csv (r'Data\credit_card_churn.csv')).apply(lambda x: pd.factorize(x)[0])


'''Credit Card Approvals'''

#target = aprroved yes-no

df_four = transform_data_bank(pd.read_csv (r'Data\clean_dataset.csv')).apply(lambda x: pd.factorize(x)[0])



#### In processing techniques comparison

## SECTION 1

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

#### INTER-MODEL EVALUATION

In this section we compare the first three in-processing mitigation techniques againts a base model:

- ThresholdOptimizer
- ExponentiatedGradient
- GridSearch 

these techniques  returns a set of the best parameters based on a selected fairness constraint.

We validate this models in the following ways:

- K-fold cross validation 
- Test with four different datasets 
- Test with four different classifier

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

END-USER REQUIREMENTS:

- Select number of split for cross validation
- Select Dataset
- Select models


1 Requirment:

- Select number of split for cross validation 

In [9]:
#Select number of KFOLDS

N_SPLIT = 2

- Select dataset from following options:

    | df_one | df_two | df_three | df_four |

In [5]:
df = df_one

# Select name of the dataset for exporting in CSV

Dataset_number = "DS_one"

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

#### Select models

The function "intermodel_to_cv" compute the following:

- Find best parameteres to improve Demographic parity within gender
- Returns and fits best parameter 
- Validate with cross validation
- Create df with performance and fairness metris returning min-mean-max values of each metric 
- Export results in csv in the following directory: 'Result_csv'

Each cell will output the folded hands emoji when each k-fold is completed 


The function box_graph will output the following:

- Generate a box plot chart containing the score for demographic parity and accuracy considering the results from min-mean-max of the validation for each mitigation technique compared to the unmitigated model

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

##### LOGISTIC REGRESSION 

In [ ]:
model = LogisticRegression(solver='liblinear', fit_intercept=True)
Model_name = "LogisticRegression"
intermodel_to_csv(N_SPLIT,model,df,Model_name,Dataset_number)

In [12]:
Model_name = "LogisticRegression"
comparison_df_logistic = pd.read_csv (f'Result_csv\model_comparison_{Model_name}_{Dataset_number}.csv' , index_col= 0)
box_df_logistic = pd.read_csv (f'Result_csv/box_df_{Model_name }_{Dataset_number}.csv' , index_col= 0)
box_df_logistic_chart = box_graph_inter(box_df_logistic )
box_df_logistic_chart

In [13]:
selection_comparison(comparison_df_logistic)

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

##### DECISION TREE 

In [12]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)


In [15]:
model =  DecisionTreeClassifier(random_state=0)
Model_name = "DecisionTree"
intermodel_to_csv(N_SPLIT,model,df,Model_name,Dataset_number)

Starting hyperparameter tunings
 - Base model 🙏 🙏 
 - Treshold optimizer 🙏 🙏 
 - Exponentiated gradient 🙏 🙏 
 - Grid search 🙏 🙏 
 exporting KNN resutls to csv


In [16]:
box_df_DecisionTree = pd.read_csv (f'Result_csv/box_df_{Model_name }_{Dataset_number}.csv' , index_col= 0)
comparison_df_DecisionTree = pd.read_csv (f'Result_csv\model_comparison_{Model_name}_{Dataset_number}.csv' , index_col= 0)
box_graph_inter(box_df_DecisionTree )

In [89]:
selection_comparison(comparison_df_DecisionTree )

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

##### RANDOM FOREST

In [ ]:
model = RandomForestClassifier(max_depth=2, random_state=0)
Model_name = "Random Forest"
intermodel_to_csv(N_SPLIT,model,df,Model_name,Dataset_number)

In [21]:
Model_name = "Random Forest"
box_df_RF = pd.read_csv (f'Result_csv/box_df_{Model_name }_{Dataset_number}.csv' , index_col= 0)
comparison_df_RF = pd.read_csv (f'Result_csv\model_comparison_{Model_name}_{Dataset_number}.csv' , index_col= 0)
box_graph_inter(box_df_RF)

In [95]:
selection_comparison(comparison_df_RF)

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

##### XGBOOST

In [ ]:
model = XGBClassifier()
Model_name = "XGB"
intermodel_to_csv(N_SPLIT,model,df,Model_name,Dataset_number)

In [17]:
Model_name = "XGB"
comparison_df_XGB = pd.read_csv (f'Result_csv\model_comparison_{Model_name}_{Dataset_number}.csv' , index_col= 0)
box_df_XGB = pd.read_csv (f'Result_csv/box_df_{Model_name }_{Dataset_number}.csv' , index_col= 0)
box_graph_inter(box_df_XGB)

In [101]:
selection_comparison(comparison_df_XGB)

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

### PARETO FRONT COMPARISON

- In this section we compared the three different techniques with the four selected model with a parereto front chart that will show the eight generated model considerding accuracy and demographic parity 

In [22]:
tuner_comparison = pd.concat([comparison_df_logistic,comparison_df_RF,comparison_df_XGB,comparison_df_DecisionTree])
tuner_comparison_pareto = px.scatter(tuner_comparison ,x= 'Accuracy score', y= 'Demographic parity difference', color = tuner_comparison.index, symbol = tuner_comparison .index, size='Accuracy score',hover_data=['Accuracy score'])
tuner_comparison_pareto

In [20]:
comparison_df_RF

,Accuracy score,False positive rate,False negative rate,F1 score,Precision score,Recall score,Selection rate,Demographic parity difference,Equalized odds difference,SR Female,SR Male,FPR Male,FPR Female,FNR Male,FNR Female
Base_Model_SVM,0.841897,0.061407,0.462941,0.620630,0.735086,0.537059,0.175947,0.182324,0.180832,0.053941,0.236265,0.091827,0.013451,0.435749,0.616581
GridSearch_SVM,0.821750,0.051184,0.579042,0.531699,0.723455,0.420958,0.140260,0.008961,0.278812,0.134264,0.143224,0.039332,0.069893,0.620903,0.342091
ExponentiatedGradient_SVM,0.824023,0.055166,0.556699,0.548064,0.718530,0.443301,0.148614,0.012414,0.278907,0.140304,0.152719,0.043215,0.074019,0.598575,0.319668
ThresholdOptimizer_SVM,0.771444,0.220913,0.253132,0.611538,0.517909,0.746868,0.347564,0.001849,0.192124,0.348798,0.346948,0.183511,0.279800,0.282039,0.089915


_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## SECTION 2

#### INTER-MODEL EVALUATION

In this section we compare the remaining three in-processing mitigation techniques:

- AdversarialDebiasing
- MetaFairClassifier
- PrejudiceRemover

these techniques  returns mitigated classifier.

We validate this models in the following ways:

- K-fold cross validation 
- Test with four different datasets 


In [35]:
intermodel_to_csv_two(N_SPLIT,df,Dataset_number)

Starting fair classifier generation
epoch 0; iter: 0; batch classifier loss: 152.051193; batch adversarial loss: 0.704417
epoch 1; iter: 0; batch classifier loss: 6.426623; batch adversarial loss: 0.662110
epoch 2; iter: 0; batch classifier loss: 6.094672; batch adversarial loss: 0.642306
epoch 3; iter: 0; batch classifier loss: 7.695511; batch adversarial loss: 0.634439
epoch 4; iter: 0; batch classifier loss: 8.580505; batch adversarial loss: 0.660938
epoch 5; iter: 0; batch classifier loss: 3.083141; batch adversarial loss: 0.647549
epoch 6; iter: 0; batch classifier loss: 17.201778; batch adversarial loss: 0.660443
epoch 7; iter: 0; batch classifier loss: 1.556460; batch adversarial loss: 0.634293
epoch 8; iter: 0; batch classifier loss: 1.753812; batch adversarial loss: 0.648838
epoch 9; iter: 0; batch classifier loss: 2.888736; batch adversarial loss: 0.593005
epoch 10; iter: 0; batch classifier loss: 1.119418; batch adversarial loss: 0.604270
epoch 11; iter: 0; batch classifier 

In [125]:
box_df_classifier= pd.read_csv (f'Result_csv/box_df_{Dataset_number}.csv' , index_col= 0)
comparison_df_classifier = pd.read_csv ('Result_csv\model_comparison_DS_one.csv' , index_col= 0)
box_df_classifier = box_graph_inter(box_df_classifier)
box_df_classifier

In [126]:
box_df_classifier= pd.read_csv (f'Result_csv/box_df_{Dataset_number}.csv' , index_col= 0)

In [127]:
box_df_classifier

,Accuracy score,False positive rate,False negative rate,F1 score,Precision score,Recall score,Selection rate,Demographic parity difference,Equalized odds difference,SR Female,SR Male,FPR Male,FPR Female,FNR Male,FNR Female,Model_name
min,0.839199,0.053359,0.411450,0.603635,0.697948,0.505242,0.161916,0.052237,0.142973,0.127054,0.179291,0.049072,0.060054,0.433024,0.290051,AdversarialDebiasing
mean,0.839901,0.067203,0.453104,0.621116,0.723784,0.546896,0.182733,0.067884,0.173784,0.137276,0.205161,0.066181,0.068763,0.479219,0.305435,AdversarialDebiasing
max,0.840602,0.081046,0.494758,0.638597,0.749621,0.588550,0.203550,0.083532,0.204594,0.147498,0.231030,0.083289,0.077472,0.525414,0.320819,AdversarialDebiasing
min,0.604238,0.273551,0.079418,0.528089,0.370237,0.835159,0.408943,0.051947,0.054116,0.305178,0.461098,0.294872,0.240756,0.088989,0.026622,MetaFairClassifier
mean,0.678447,0.384754,0.122129,0.573787,0.431289,0.877871,0.503519,0.103934,0.102956,0.469115,0.521101,0.366415,0.415255,0.128002,0.089436,MetaFairClassifier
max,0.752656,0.495956,0.164841,0.619484,0.492340,0.920582,0.598096,0.155920,0.151796,0.633052,0.581105,0.437958,0.589754,0.167015,0.152249,MetaFairClassifier
min,0.808784,0.063770,0.580653,0.506578,0.662233,0.407341,0.146560,0.137922,0.110383,0.046696,0.191910,0.090071,0.018016,0.556323,0.686441,PrejudiceRemover
mean,0.808790,0.065776,0.586656,0.510048,0.665987,0.413344,0.149473,0.148163,0.136113,0.050342,0.198505,0.094876,0.019926,0.566190,0.702304,PrejudiceRemover
max,0.808796,0.067783,0.592659,0.513518,0.669740,0.419347,0.152386,0.158404,0.161843,0.053988,0.205100,0.099681,0.021835,0.576058,0.718166,PrejudiceRemover


In [111]:
selection_comparison(comparison_df_classifier)

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

### PARETO FRONT COMPARISON

In [91]:

classifier_parerto = px.scatter(comparison_df_classifier,x= 'Accuracy score', y= 'Demographic parity difference', color = comparison_df_classifier.index, symbol = comparison_df_classifier.index, size='Accuracy score',hover_data=['Accuracy score'])

classifier_parerto .show()

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## SECTION 3

### OVERALL COMPARISON 

In [90]:
#comparison_df_classifier = comparison_df_classifier.drop(columns=['Model_name'])
total_comparison = pd.concat([comparison_df_classifier,tuner_comparison])
total_comparison = total_comparison.drop('Model_name', axis=1)
# Create scatter plot for model comparison 


In [93]:

total_comparison_pareto = px.scatter(total_comparison,x= 'Accuracy score', y= 'Demographic parity difference', color = total_comparison.index, symbol = total_comparison.index,  size='Accuracy score',hover_data=['Accuracy score'])

total_comparison_pareto.show()

In [89]:
import plotly.express as px
df = px.data.iris()
threed_chart = px.scatter_3d(total_comparison, x='Equalized odds difference', y='Accuracy score', z='Demographic parity difference', color = total_comparison.index)
threed_chart.show()

_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________